# Symétries de KTlO2 

In [1]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
mp_key = "txVH20lqfEIM4Yxb2k5ptbih3BpgcIsN"
mp_id = "mp-8175"

In [3]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

## Inversion sur O

In [105]:
i_atom = 7
i_symmop = 2

In [106]:
view = quick_view(prim_struc, "packed", conventional = True)
display(view)

view.script('select potassium; color white')
view.script('select thallium; color pink')
view.script('select oxygen; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [107]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]


In [108]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print("Position initale de l'atome d'oxygène :",pos_init)

Position initale de l'atome d'oxygène : [0.         0.         0.73208426]


In [109]:
pos_final = symmop.operate(pos_init)
print("Position finale de l'atome d'oxygène après transformation :", pos_final)

Position finale de l'atome d'oxygène après transformation : [ 0.          0.         -0.73208426]


In [110]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print("Indice de l'atome le plus proche de la position finale:",j_atom,"\n","Ses coordonées exactes :" ,conv_struc.sites[j_atom -1].frac_coords)

Indice de l'atome le plus proche de la position finale: 10 
 Ses coordonées exactes : [0.         0.         0.26791574]


La première opération de symétrie est la suivante : (x,y,z) $\rightarrow$ (-x,-y,-z) 

Elle est définie par une matrice de rotation (Rot) $\begin{pmatrix}-1&0&0\\0&-1&0\\0&0&-1\end{pmatrix}$ et une translation (tau) nulle.

C'est donc bien une inversion. Une fois appliquée à l'atome d'oxygène qui se trouve initialement à la position $\begin{pmatrix} 0. & 0. & 0.73208426 \end{pmatrix}$ , on arrive alors à la position $\begin{pmatrix}0. & 0. & -0.73208426\end{pmatrix}$

## Rotation de 90° autour de z sur Tl

In [111]:
i_atom = 5
i_symmop = 8

In [112]:
view = quick_view(prim_struc, "packed", conventional = True)
display(view)

view.script('select potassium; color white')
view.script('select thallium; color pink')
view.script('select oxygen; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [113]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]


In [114]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print("Position initale de l'atome de thallium : ",pos_init)

Position initale de l'atome de thallium :  [0.66666667 0.33333333 0.33333333]


In [115]:
pos_final = symmop.operate(pos_init)
print("Position finale de l'atome de thallium après transformation :", pos_final)

Position finale de l'atome de thallium après transformation : [-0.33333333 -0.66666667  0.33333333]


In [116]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print("Indice de l'atome le plus proche de la position finale:",j_atom,"\n","Ses coordonées exactes :" ,conv_struc.sites[j_atom -1].frac_coords)

Indice de l'atome le plus proche de la position finale: 5 
 Ses coordonées exactes : [0.66666667 0.33333333 0.33333333]


La seconde opération de symétrie est la suivante : (x,y,z) $\rightarrow$ (-y,-x,z) 

Elle est définie par une matrice de rotation (Rot) $\begin{pmatrix}0&-1&0\\-1&0&0\\0&0&1\end{pmatrix}$ et une translation (tau) nulle.

C'est donc bien une rotation de 90° autour de l'axe z, c'est-à-dire une $C_4$ selon l'axe $[001]$. Une fois appliquée à l'atome de  thallium qui se trouve initialement à la position $\begin{pmatrix}0.66666667 & 0.33333333 & 0.33333333\end{pmatrix}$, on arrive alors à la position $\begin{pmatrix} -0.33333333 &-0.66666667 & 0.33333333\end{pmatrix}$

## Réflexion avec Plan miroir sur K

In [123]:
i_atom = 1
i_symmop = 7

In [128]:
view = quick_view(prim_struc, "packed", conventional = True)
display(view)

view.script('select potassium; color white')
view.script('select thallium; color pink')
view.script('select oxygen; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [129]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]


In [130]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print("Position initale de l'atome de potassium' : ",pos_init)

Position initale de l'atome de potassium' :  [0.66666667 0.33333333 0.83333333]


In [131]:
pos_final = symmop.operate(pos_init)
print("Position finale de l'atome de potassium après transformation :", pos_final)

Position finale de l'atome de potassium après transformation : [ 0.33333333  0.66666667 -0.83333333]


In [132]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print("Indice de l'atome le plus proche de la position finale:",j_atom,"\n","Ses coordonées exactes :" ,conv_struc.sites[j_atom -1].frac_coords)

Indice de l'atome le plus proche de la position finale: 5 
 Ses coordonées exactes : [0.66666667 0.33333333 0.33333333]


La troisème opération de symétrie est la suivante : (x,y,z) $\rightarrow$ (y,x,-z) 

Elle est définie par une matrice de rotation (Rot) $\begin{pmatrix}0&1&0\\1&0&0\\0&0&-1\end{pmatrix}$ et une translation (tau) nulle.

Les coodornées en z changent de signe ce qui montre qu'il y a une réflexion selon le plan miroir xy (plan perpendiculaire à la direction $[001]$ ), combinée avec une permutation de x et y. Une fois appliquée à l'atome de potassium qui se trouve initialement à la position $\begin{pmatrix}0.66666667 & 0.33333333 & 0.83333333\end{pmatrix}$, on arrive alors à la position $\begin{pmatrix}0.33333333 & 0.66666667 & -0.83333333 \end{pmatrix}$

In [133]:
#aide pour trouver les index des symétries

for i, symmop in enumerate(symmops):
    print(f"Index {i+1}: {symmop}")

Index 1: Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0. 0. 0.]
Index 2: Rot:
[[-1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Index 3: Rot:
[[ 0. -1.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Index 4: Rot:
[[ 0.  1.  0.]
 [-1.  1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Index 5: Rot:
[[-1.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Index 6: Rot:
[[ 1. -1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Index 7: Rot:
[[ 0.  1.  0.]
 [ 1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Index 8: Rot:
[[ 0. -1.  0.]
 [-1.  0.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Index 9: Rot:
[[ 1. -1.  0.]
 [ 0. -1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Index 10: Rot:
[[-1.  1.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Index 11: Rot:
[[-1.  0.  0.]
 [-1.  1.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]
Index 12: Rot:
[[ 1.  0.  0.]
 [ 1. -1.  0.]
 [ 0.  0.  1.]]
tau
[0. 0. 0.]
Index 13: Rot:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
tau
[0.66666667 0.33333333 0.33333333]
Index 14: Rot:
